# Klassifikation 

In diesem Notebook trainieren wir einen Classifier. Aus Beispielen lernt der Classifier, zu welcher Klasse ein Element gehört, und kann neue Elemente mit den gelernten Regeln einordnen. 

Wir lesen zunächst Dokumente aus 4 Wikipedia-Kategorien ein.

### Texten sammeln (API)

In [94]:
pip install -U scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/f1/7d/2e562207176a5dcdad513085670674bb11ffaf37e1393eacb6d7fb502481/scikit_learn-1.3.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/06/15/e73734f9170b66c6a84a0bd7e03586e87e77404e2eb8e34749fc49fa43f7/scipy-1.11.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ---------------------------------------- 59.1/59.1 kB 3.1 MB/s eta 0:00:00
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
   -- ------------------------------------- 0.6/9.2 MB 12.0 MB/s eta 0:00:01
   -- ------------------------------------- 0.6/9.2 MB

In [48]:
import SaveWiki

#SaveWiki.downloadWikiCat('Liste_(Parteien)','partei')
#SaveWiki.downloadWikiCat('Liste_von_Religionen_und_Weltanschauungen','religion')







##### Texten aufmachen und lesen

In [62]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [73]:
bsptext= readtext('religion/Liste_von_Religionen_und_Weltanschauungen.txt')
print(bsptext[:300])


Eine Reihe der Religionen und Weltanschauungen der Welt lässt sich schwer systematisieren, da vielfältige Elemente ineinanderspielen und es unterschiedliche Auffassungen dazu gibt, was eine Religion oder eine Weltsicht ausmacht (mit diesem Thema beschäftigt sich unter anderem die Religionswissenscha


### Merkmalen des Textes auswählen

*1.stopwords ausschließen*

*Zeichen ausschließen*

*closed class words ausschließen*

In [74]:
#1:
import nltk
stopwords = nltk.corpus.stopwords.words('german')
print(stopwords)

['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'anderes', 'anderm', 'andern', 'anderr', 'anders', 'auch', 'auf', 'aus', 'bei', 'bin', 'bis', 'bist', 'da', 'damit', 'dann', 'der', 'den', 'des', 'dem', 'die', 'das', 'dass', 'daß', 'derselbe', 'derselben', 'denselben', 'desselben', 'demselben', 'dieselbe', 'dieselben', 'dasselbe', 'dazu', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'denn', 'derer', 'dessen', 'dich', 'dir', 'du', 'dies', 'diese', 'diesem', 'diesen', 'dieser', 'dieses', 'doch', 'dort', 'durch', 'ein', 'eine', 'einem', 'einen', 'einer', 'eines', 'einig', 'einige', 'einigem', 'einigen', 'einiger', 'einiges', 'einmal', 'er', 'ihn', 'ihm', 'es', 'etwas', 'euer', 'eure', 'eurem', 'euren', 'eurer', 'eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten', 'hier', 'hin', 'hinter', 'ich', 'mich', 'mir', 'ihr', 'ihre', 'ihrem', 'ihren', 'ihrer', 'ihres', 'euc

In [77]:
import re
from HanTa import HanoverTagger as ht
from collections import Counter

tagger = ht.HanoverTagger('morphmodel_ger.pgz')

def closed_class(pos):
    if pos[0] == '$':
        return True
    elif pos in ["APPR", "APPRART", "APPO", "APZR", "ART", "KOUI", "KOUS", "KON", "KOKOM", "PDS", "PDAT", "PIS", "PIAT", "PIDAT", "PPER", "PPOSS", "PPOSAT", "PRELS", "PRELAT", "PRF", "PWS", "PWAT", "PWAV", "PAV", "PTKZU", "PTKNEG", "VAFIN", "VAIMP", "VAINF", "VAPP", "VMFIN", "VMINF", "VMPP"]:
        return True
    return False

def features_from_text(text):
    wordcounts = Counter()
    tlen = 0
    
    satzliste =  nltk.sent_tokenize(text,language='german')
    for satz in satzliste:
        tokens =  nltk.word_tokenize(satz,language='german')
        tokens = [lemma for (word,lemma,pos) in tagger.tag_sent(tokens) if not closed_class(pos)]
        tokens = [t for t in tokens if t.lower() not in stopwords]
        tokens = [t for t in tokens if re.search('^\w+$',t)]
        tlen += len(tokens)
        wordcounts.update(tokens)

    return {w:wordcounts[w]/tlen for w in wordcounts}

In [78]:
len(features_from_text(bsptext))

533

Wir versuchen aus jede Klasse 50 Dokumente zu lesen, die nicht extrem kurz oder lang sind.

In [81]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        dirsize = 0
        for file in glob.glob('data_intocode/'+directory+"/*.txt"):
            text = readtext(file)
            if len(text) > 500 and len(text) < 10000:
                docs.append((features_from_text(text),directory))
                dirsize += 1
            if dirsize >= 50:
                break
    return docs

data=read_data(['religion','partei'])

In [89]:
data[1]

({'Enkoimesis': 0.0055248618784530384,
  'altgriechisch': 0.0055248618784530384,
  'Ἐγκοίμησις': 0.0055248618784530384,
  'Inkubation': 0.011049723756906077,
  'lateinisch': 0.0055248618784530384,
  'Incubatio': 0.0055248618784530384,
  'Tempelschlaf': 0.016574585635359115,
  'bezeichnen': 0.0055248618784530384,
  'Antike': 0.0055248618784530384,
  'belegt': 0.0055248618784530384,
  'Praxis': 0.011049723756906077,
  'Trauminkubation': 0.0055248618784530384,
  'Kranke': 0.0055248618784530384,
  'Heiligtum': 0.022099447513812154,
  'Gott': 0.011049723756906077,
  'Heros': 0.0055248618784530384,
  'aufsuchen': 0.0055248618784530384,
  'manchmal': 0.0055248618784530384,
  'Verbindung': 0.0055248618784530384,
  'entsprechend': 0.0055248618784530384,
  'Ritual': 0.0055248618784530384,
  'mehr': 0.0055248618784530384,
  'minder': 0.0055248618784530384,
  'aufwändig': 0.0055248618784530384,
  'Vorbereitung': 0.0055248618784530384,
  'Bad': 0.0055248618784530384,
  'Fast': 0.0055248618784530384

Wir mischen die Daten und teilen in Test- und Trainingsdaten

In [83]:
len(data)

63

In [84]:
import random

random.shuffle(data)
train_data = data[:40]
test_data = data[40:]

Wir schauen uns jetzt mal ein Dokument an:

In [85]:
len(train_data[27])

2

Wir haben jetzt für jedes Dokument einene Merkmalsvektor. Man merke: der Vektor ist eigentlich so lang wie die Anzahl der unterschiedlichen Wörter in der ganzen Sammlung. Alle Wörter, die nicht erwähnt werden haben den Wert 0. 

Wörter, die nur in ein oder zwei Dokumementen vorkommen sind, für die Klassifikation nicht besonders nützlich. 
Wir nutzen nachher nur die Wörter, die in mindestens 5 Dokumente vorkommen. Um das vorzubereiten, berechnen wir für alle Wörter die Dokumentfrequenz>

In [86]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

In [90]:
#docfreq

Textvektor=[5,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

## Klassifikation mit Scikit Learn

Die Bibliothek Scikit Learn stellt verschiedene Klassifikationsmodelle zur Verfügung. Wir müssen jetzt richtige Merkmalsvektoren aufbauen, bei denen die Position die Bedeutung einer Zahl bestimmt. Dazu machen wir erst eine feste Liste mit allen Wörtern. Wörter, die zu selten sind lassen wir weg.

In [95]:
from sklearn import linear_model, datasets


allfeatures = [w for w in docfreq if docfreq[w] > 4]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

Wir schauen uns mal einen Vektor an:

In [96]:
len(train_vec[20])

122

In [97]:
train_vec[20]

[0.0,
 0.006920415224913495,
 0.03460207612456748,
 0.0,
 0.0,
 0.0034602076124567475,
 0.006920415224913495,
 0.0,
 0.01384083044982699,
 0.0034602076124567475,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0034602076124567475,
 0.0,
 0.006920415224913495,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.01384083044982699,
 0.0,
 0.0,
 0.01384083044982699,
 0.0,
 0.006920415224913495,
 0.0,
 0.0034602076124567475,
 0.0,
 0.0,
 0.0,
 0.0034602076124567475,
 0.0034602076124567475,
 0.0,
 0.0,
 0.0,
 0.0034602076124567475,
 0.0034602076124567475,
 0.0,
 0.0,
 0.010380622837370242,
 0.0,
 0.0,
 0.0,
 0.0,
 0.10726643598615918,
 0.0034602076124567475,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0034602076124567475,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0034602076124567475,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006920415224913495,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0034602076124567475,
 0.0034602076124567475,
 0.00346

In [98]:
train_label[20]

'partei'

Ein einfaches, aber effektives Klassifikationsmodell, das meistens gute Ergebnisse liefert, ist die logistische Regression.

In [99]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True)
logreg.fit(train_vec,train_label)

LogisticRegression(C=1000000000.0, verbose=True)

Wir können jetzt mit dem trainierten Classifier ein neues Dokument klassifizieren:

In [100]:
v = make_feat_vec(test_data[17][0],allfeatures) 
logreg.predict([v])

array(['partei'], dtype='<U8')

Richtig ist:

In [101]:
test_data[17][1]

'partei'

Oder direkt für alles Testdaten:

In [102]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

In [103]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

100.0 Prozent korrekt


### Evaluation

f-Faktor
Recall
Precision
Stearman

Wo sind aber die Fehler aufgetreten? Un diese Frage zu beantworten, ist sogenannte eine _Konfusionsmatrix_ sehr hilfreich. NLTK hat eine Funktion zum erstellen eines Konfusionsmatrizen:

In [104]:
cm = nltk.ConfusionMatrix(test_label, pred_label)
print(cm)

         |     r |
         |     e |
         |  p  l |
         |  a  i |
         |  r  g |
         |  t  i |
         |  e  o |
         |  i  n |
---------+-------+
  partei |<18> . |
religion |  . <5>|
---------+-------+
(row = reference; col = test)



Es gibt verschiedene andere Klassifikatorn, die wie nutzen können:

In [105]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_vec,train_label)
pred_label = list(knn.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

87.0 Prozent korrekt


In [106]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(train_vec,train_label)
pred_label = list(rf.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

87.0 Prozent korrekt
